In [1]:
import pandas as pd
import plotly.graph_objects as go

In [2]:
data = pd.read_csv("./cleaned/usdrub.csv")

In [3]:
data.drop(columns=["Unnamed: 0"], inplace=True)


In [4]:
data.head()

,date,open,high,low,close,vol
0,2014-02-06,34.8934,34.9337,34.5848,34.7102,7669
1,2014-02-07,34.7300,34.8300,34.5300,34.7500,10
2,2014-05-05,35.7747,35.8635,35.6700,35.7100,1564
3,2014-10-07,39.7165,39.9751,39.6550,39.8873,376
4,2014-10-08,39.8959,40.1560,39.6877,39.9721,901


In [12]:
data['ew'] = data['close'].ewm(alpha=0.1).mean()
data.head()

,date,open,high,low,close,vol,ew
0,2014-02-06,34.8934,34.9337,34.5848,34.7102,7669,34.710200
1,2014-02-07,34.7300,34.8300,34.5300,34.7500,10,34.731147
2,2014-05-05,35.7747,35.8635,35.6700,35.7100,1564,35.092348
3,2014-10-07,39.7165,39.9751,39.6550,39.8873,376,36.486634
4,2014-10-08,39.8959,40.1560,39.6877,39.9721,901,37.337765


In [22]:
fig = go.Figure(data=[go.Candlestick(x=data['date'],
                open=data['open'],
                high=data['high'],
                low=data['low'],
                close=data['close']),
                go.Scatter(mode="lines", x=data['date'], y=data["ew"])])
fig.layout.yaxis.fixedrange = False
fig.show()